In [1]:
import pandas as pd
from math import isnan
import duckdb
pd.set_option("display.precision", 3)
pd.set_option("display.max_columns", 30)

In [2]:
from utils.betting import (
	pivot_lines,
	value_count_plus,
	get_results,
	get_pp_close,
)

In [3]:
pp = get_pp_close()
pp = pp.loc[pp.league_name == 'NFL'].copy(deep=True).reset_index(drop=True)

In [4]:
con = duckdb.connect("C:/Github/nfl_sim/data/nfl.duckdb")

In [5]:
con.execute("select * from weekly where team = 'MIA'").df().tail(10)

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,team,opponent_team,completions,attempts,passing_yards,passing_tds,...,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance,fantasy_points,fantasy_points_ppr
13394,00-0039040,D.Achane,De'Von Achane,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,18.2,26.2
13395,00-0039880,M.Washington,Malik Washington,WR,WR,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,8.5,9.5
13396,00-0039899,P.Paul,Patrick Paul,OT,OL,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0.0,0.0
13397,00-0039911,C.Robinson,Chop Robinson,LB,LB,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0.0,0.0
13398,00-0040127,K.Grant,Kenneth Grant,DT,DL,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0.0,0.0
13399,00-0040175,Jo.Phillips,Jordan Phillips,DT,DL,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0.0,0.0
13400,00-0040178,J.Marshall,Jason Marshall Jr.,CB,DB,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0.0,0.0
13401,00-0040182,D.Trader,Dante Trader Jr.,SAF,DB,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0.0,0.0
13402,00-0040198,O.Gordon,Ollie Gordon II,RB,RB,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,1.3,2.3
13403,00-0040742,J.Savaiinaea,Jonah Savaiinaea,OT,OL,https://static.www.nfl.com/image/upload/f_auto...,2025,2,REG,MIA,NE,0,0,0,0,...,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0.0,0.0


In [6]:
weekly = con.execute("""select * from weekly left join schedule on weekly.season = schedule.season and weekly.week = schedule.week and (away_team = team or home_team = team)
                      where schedule.season > 2020 and position in ('WR',
 'RB',
 'TE',
 'QB',
 'P',
 'K',
 'FB',
 'T',
 'HB',
 'LB',
 'G',
 'WR/R',
 'RB/W',
 'C')""").df()
con.close()
games = weekly.copy()
sorted(list(games.columns))

['air_yards_share',
 'attempts',
 'away_coach',
 'away_moneyline',
 'away_qb_id',
 'away_qb_name',
 'away_rest',
 'away_score',
 'away_spread_odds',
 'away_team',
 'carries',
 'completions',
 'def_fumbles',
 'def_fumbles_forced',
 'def_interception_yards',
 'def_interceptions',
 'def_pass_defended',
 'def_qb_hits',
 'def_sack_yards',
 'def_sacks',
 'def_safeties',
 'def_tackle_assists',
 'def_tackles_for_loss',
 'def_tackles_for_loss_yards',
 'def_tackles_solo',
 'def_tackles_with_assist',
 'def_tds',
 'div_game',
 'espn',
 'fantasy_points',
 'fantasy_points_ppr',
 'fg_att',
 'fg_blocked',
 'fg_blocked_distance',
 'fg_blocked_list',
 'fg_long',
 'fg_made',
 'fg_made_0_19',
 'fg_made_20_29',
 'fg_made_30_39',
 'fg_made_40_49',
 'fg_made_50_59',
 'fg_made_60_',
 'fg_made_distance',
 'fg_made_list',
 'fg_missed',
 'fg_missed_0_19',
 'fg_missed_20_29',
 'fg_missed_30_39',
 'fg_missed_40_49',
 'fg_missed_50_59',
 'fg_missed_60_',
 'fg_missed_distance',
 'fg_missed_list',
 'fg_pct',
 'ftn',


In [7]:
con.close()

In [8]:
pp2nfl = {
 'FG': 'fg_made',
 'INT': 'passing_interceptions',
 'PAT Made': 'pat_made',
 'Pass Attempts': 'attempts',
 'Pass Completions': 'completions',
'Pass Deflections': 'def_pass_defended',
 'Pass TDs': 'passing_tds',
 'Pass Yards': 'pass_yards',
 'Rec Targets': 'targets',
 'Receiving Yards': 'receiving_yards',
 'Receptions': 'receptions',
 'Rush TDs': 'rushing_tds',
 'Rush Yards': 'rushing_yards', 
 'Sacks': 'def_sacks',
 'Sacks Taken': 'sacks_suffered',
}

In [9]:
pp['stat_nfl'] = pp['stat'].apply(lambda x: pp2nfl.get(x,x))
games = games.rename({'player_display_name':'player', 'opponent_team':'opp', 'gameday':'date'},axis=1)
games['date'] = pd.to_datetime(games['date'])

In [10]:
games['game_line'] = games.apply(lambda x: x['spread_line'] if x['home_team'] == x['team'] else -x['spread_line'],axis=1)
games['Rush+Rec TDs'] = games['receiving_tds'] + games['rushing_tds']
games['Completion Percentage'] = 100 * (games['completions'] / games['attempts'])
games['Pass+Rush Yds'] = games['passing_yards'] + games['rushing_yards']
games['Rush+Rec Yds'] = games['receiving_yards'] + games['rushing_yards']
games['Kicking Points'] = games['pat_made'] + (3 * games['fg_made'])
games['Fantasy Score'] = ((games['passing_yards'] * 0.04) + (games['passing_tds'] * 4) + (2 * -games['passing_interceptions']) + (games['rushing_yards'] * 0.1) + (games['receiving_yards'] * 0.1) + #missing 2 pt conv
                    (games['receptions']) + (2 * (-games['receiving_fumbles_lost'] - games['rushing_fumbles_lost'] )) + (games['Rush+Rec TDs'] * 6) + 
                    games['pat_made'] + (3 * games['fg_made']) + games['fg_made_40_49'] + (games['fg_made_50_59'] * 2) + (games['fg_made_60_'] * 3) - games['pat_missed']  - games['fg_missed'] )

In [11]:
games[['player','date','team','opp']].sort_values(by='date')

,player,date,team,opp
8267,Ronald Jones,2021-09-09,TB,DAL
8253,Tony Pollard,2021-09-09,DAL,TB
8240,Greg Zuerlein,2021-09-09,DAL,TB
8251,Cedrick Wilson Jr.,2021-09-09,DAL,TB
8250,Dalton Schultz,2021-09-09,DAL,TB
...,...,...,...,...
40767,Rico Dowdle,2025-09-14,CAR,ARI
40766,Christian Rozeboom,2025-09-14,CAR,ARI
40765,Hunter Renfrow,2025-09-14,CAR,ARI
40779,Tetairoa McMillan,2025-09-14,CAR,ARI


In [12]:
ref2pp = {
    'GNB':'GB',
    'JAX':'JAC',
    'KAN':'KC',
    'LVR':'LV',
    'NOR':'NO',
    'NWE':'NE',
    'SFO':'SF',
    'TAM':'TB',
}
games['team'] = games['team'].apply(lambda x: ref2pp.get(x,x))


def get_results(lines, games):
	def calc_outcome(row):
		outcome = 0
		for part in row['stat_nfl'].split('+afsda'):
			try:
				outcome += row[part]
			except KeyError:
				return float('nan')
		return outcome

	comb = lines.merge(games, on=['player', 'date', 'team', 'opp'])
	comb['result'] = comb.apply(lambda row: calc_outcome(row), axis=1)
	comb['o_u'] = [
		'under' if x > y else 'over' if x < y else 'push' if x == y else 'N/A'
		for x, y in zip(comb['line'], comb['result'])
	]
	return comb.loc[comb.o_u != 'N/A'].reset_index(drop=True)

df = get_results(pp, games)
df.sample(3)

,player,team,line,alt_line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,index,...,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,game_line,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Kicking Points,Fantasy Score,o_u
8168,Brenton Strange,JAC,39.5,demon,Receiving Yards,HOU,9.0,NFL,2024-09-29 11:00:00-06:00,180073.0,2024-09-29,2024-09-29 10:40:00-06:00,712e6510-8,48668cc3-2,NaN,...,Trevor Lawrence,C.J. Stroud,Doug Pederson,DeMeco Ryans,Tra Blake,HOU00,NRG Stadium,-5.5,0,NaN,0,19,0,4.9,under
118,Ja'Marr Chase,CIN,65.5,standard,Receiving Yards,MIN,9.0,NFL,NaT,150731.0,2023-12-16,2023-12-15 14:14:00-06:00,04ab37ff-3,066f7631-f,565.0,...,Nick Mullens,Jake Browning,Kevin O'Connell,Zac Taylor,Bill Vinovich,CIN00,Paycor Stadium,3.0,0,NaN,0,64,0,10.4,under
8878,Justin Jefferson,MIN,16.5,standard,Fantasy Score,NYJ,9.0,NFL,2024-10-06 07:00:00-06:00,211229.0,2024-10-06,2024-10-06 07:10:00-06:00,f83cac9f-1,32c83156-1,NaN,...,Aaron Rodgers,Sam Darnold,Robert Saleh,Kevin O'Connell,Scott Novak,LON02,Tottenham Stadium,2.5,0,NaN,0,92,0,15.2,under


In [13]:
df.loc[(df.stat_nfl == 'def_sacks')][['line', 'o_u']].value_counts().reset_index()

,line,o_u,count
0,0.25,under,144
1,0.25,over,105
2,0.50,under,54
3,1.50,under,12
4,0.50,over,8
5,0.75,under,7
6,0.50,push,3
7,0.75,over,2
8,1.00,under,2
9,1.50,over,1


In [15]:
df.loc[df.date > '2025-01-01'].groupby(['stat_nfl', 'alt_line'])['o_u'].value_counts().reset_index()
res = pivot_lines(df, ['stat_nfl', 'alt_line',  'position']).sort_values(by='over_ratio').copy()
res.loc[res.alt_line=='standard']

o_u,stat_nfl,alt_line,position,over,push,under,over_ratio,count
24,Rush+Rec TDs,standard,TE,0.0,0.0,6.0,0.312,6.0
93,rushing_yards,standard,WR,32.0,0.0,48.0,0.411,80.0
44,def_sacks,standard,LB,19.0,1.0,27.0,0.429,47.0
3,Fantasy Score,standard,K,95.0,4.0,125.0,0.435,224.0
98,targets,standard,RB,161.0,23.0,205.0,0.441,389.0
91,rushing_yards,standard,RB,305.0,0.0,386.0,0.442,691.0
5,Fantasy Score,standard,RB,96.0,0.0,122.0,0.443,218.0
25,Rush+Rec TDs,standard,WR,25.0,0.0,32.0,0.448,57.0
90,rushing_yards,standard,QB,221.0,0.0,270.0,0.451,491.0
100,targets,standard,WR,481.0,61.0,577.0,0.455,1119.0


In [19]:
qb_line = df.loc[(df.stat=='Pass Yards') & (df.alt_line=='standard'), ['team','line','event_id', 'o_u']]
qb_line.columns = ['team','qb_yards', 'event_id', 'qb_res']
df = df.merge(qb_line,how='left')
df

,player,team,line,alt_line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,index,...,away_coach,home_coach,referee,stadium_id,stadium,game_line,Rush+Rec TDs,Completion Percentage,Pass+Rush Yds,Rush+Rec Yds,Kicking Points,Fantasy Score,o_u,qb_yards,qb_res
0,Brandin Cooks,DAL,24.5,goblin,Receiving Yards,BUF,9.0,NFL,NaT,152606.0,2023-12-17,2023-12-14 11:44:00-06:00,218629a7-2,fe8985b8-0,5014.0,...,Mike McCarthy,Sean McDermott,Craig Wrolstad,BUF00,New Era Field,-2.0,0,NaN,0,10,0,3.00,under,NaN,NaN
1,Quentin Johnston,LAC,2.5,standard,Receptions,LV,9.0,NFL,NaT,169519.0,2023-12-14,2023-12-14 12:14:00-06:00,478dad6b-9,bf7935e5-0,36184.0,...,Brandon Staley,Antonio Pierce,Ron Torbert,VEG00,Allegiant Stadium,-3.0,1,NaN,0,23,0,10.30,under,NaN,NaN
2,Davante Adams,LV,99.5,demon,Receiving Yards,LAC,9.0,NFL,NaT,150706.0,2023-12-14,2023-12-14 12:14:00-06:00,aa7f478b-a,bf7935e5-0,36923.0,...,Brandon Staley,Antonio Pierce,Ron Torbert,VEG00,Allegiant Stadium,3.0,1,NaN,0,101,0,24.10,over,NaN,NaN
3,Daniel Carlson,LV,6.5,standard,Kicking Points,LAC,9.0,NFL,NaT,153528.0,2023-12-14,2023-12-14 12:44:00-06:00,4426201e-1,bf7935e5-0,36921.0,...,Brandon Staley,Antonio Pierce,Ron Torbert,VEG00,Allegiant Stadium,3.0,0,NaN,0,0,9,9.00,over,NaN,NaN
4,Jakobi Meyers,LV,0.5,demon,Rush+Rec TDs,LAC,9.0,NFL,NaT,153028.0,2023-12-14,2023-12-14 12:44:00-06:00,3278565e-1,bf7935e5-0,36909.0,...,Brandon Staley,Antonio Pierce,Ron Torbert,VEG00,Allegiant Stadium,3.0,1,100.000,12,32,0,15.68,over,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30710,Terry McLaurin,WAS,0.5,demon,Rush+Rec TDs,GB,9.0,NFL,2025-09-11 18:00:00-06:00,211279.0,2025-09-11,2025-09-11 18:10:00-06:00,0f716719-c,b4a8baf3-4,NaN,...,Dan Quinn,Matt LaFleur,Brad Rogers,GNB00,Lambeau Field,-3.5,0,NaN,0,48,0,9.80,under,NaN,NaN
30711,Jayden Daniels,WAS,0.5,standard,INT,GB,9.0,NFL,2025-09-11 18:00:00-06:00,210778.0,2025-09-11,2025-09-11 18:10:00-06:00,c66ba6e2-e,b4a8baf3-4,NaN,...,Dan Quinn,Matt LaFleur,Brad Rogers,GNB00,Lambeau Field,-3.5,0,57.143,217,17,0,17.70,under,NaN,NaN
30712,Austin Ekeler,WAS,14.5,goblin,Receiving Yards,GB,9.0,NFL,2025-09-11 18:00:00-06:00,211074.0,2025-09-11,2025-09-11 18:10:00-06:00,14c1b78a-a,b4a8baf3-4,NaN,...,Dan Quinn,Matt LaFleur,Brad Rogers,GNB00,Lambeau Field,-3.5,0,NaN,17,24,0,4.40,under,NaN,NaN
30713,Tucker Kraft,GB,0.5,demon,Rush+Rec TDs,WAS,9.0,NFL,2025-09-11 18:00:00-06:00,212129.0,2025-09-11,2025-09-11 18:10:00-06:00,cf93fafa-8,b4a8baf3-4,NaN,...,Dan Quinn,Matt LaFleur,Brad Rogers,GNB00,Lambeau Field,3.5,1,NaN,0,124,0,24.40,over,NaN,NaN


In [20]:
games.iloc[5].to_dict()

{'player_id': '00-0036940',
 'player_name': 'O.Oweh',
 'player': 'Odafe Oweh',
 'position': 'LB',
 'position_group': 'LB',
 'headshot_url': 'https://static.www.nfl.com/image/upload/{formatInstructions}/league/gbmze8wfzqj5z2tqrkmq',
 'season': 2021,
 'week': 1,
 'season_type': 'REG',
 'team': 'BAL',
 'opp': 'LV',
 'completions': 0,
 'attempts': 0,
 'passing_yards': 0,
 'passing_tds': 0,
 'passing_interceptions': 0,
 'sacks_suffered': 0,
 'sack_yards_lost': 0,
 'sack_fumbles': 0,
 'sack_fumbles_lost': 0,
 'passing_air_yards': 0,
 'passing_yards_after_catch': 0,
 'passing_first_downs': 0,
 'passing_epa': nan,
 'passing_cpoe': nan,
 'passing_2pt_conversions': 0,
 'pacr': nan,
 'carries': 0,
 'rushing_yards': 0,
 'rushing_tds': 0,
 'rushing_fumbles': 0,
 'rushing_fumbles_lost': 0,
 'rushing_first_downs': 0,
 'rushing_epa': nan,
 'rushing_2pt_conversions': 0,
 'receptions': 0,
 'targets': 0,
 'receiving_yards': 0,
 'receiving_tds': 0,
 'receiving_fumbles': 0,
 'receiving_fumbles_lost': 0,
 '

In [21]:
from math import floor
def get_tiers(value):
    if isnan(value):
        return value
    else:
        return floor(value/10) * 10
    
def tiers(value):
    if isnan(value):
        return value
    if value <= -5.5:
        return 'big L'
    if value <= 5.5:
        return 'even'
    else:
        return 'big W'



df['favored'] = df['game_line'].apply(lambda x: tiers(x))
#df['bucket'] = df['yard_pct'].apply(lambda x: get_tiers(x))
subset = df.loc[(df.alt_line !='goblin') & (df['date'] > '2024-10-01')].copy()
result = pivot_lines(subset, ['alt_line', 'favored', 'stat', 'position']).sort_values(by='over_ratio', ascending=False)
result.loc[(result['count'] > 20) & (result['alt_line']=='standard')].sort_values(by=['favored','over_ratio'],ascending=False).drop(['over','push','under'],axis=1)

o_u,alt_line,favored,stat,position,over_ratio,count
146,standard,even,Completion Percentage,QB,0.589,46.0
158,standard,even,Pass TDs,QB,0.575,77.0
154,standard,even,Kicking Points,K,0.561,223.0
166,standard,even,Receiving Yards,TE,0.551,247.0
147,standard,even,FG,K,0.534,168.0
...,...,...,...,...,...,...
79,standard,big L,Fantasy Score,K,0.417,40.0
94,standard,big L,Receiving Yards,RB,0.393,112.0
81,standard,big L,Fantasy Score,RB,0.382,24.0
106,standard,big L,Rush+Rec Yds,RB,0.379,122.0


In [22]:
pp = get_pp_close()
pp = pp.loc[(pp.event_time >= pd.Timestamp.now('UTC')) & (pp.league_name == 'NFL') & (pp.scrape_time == max(pp['scrape_time']))]
pp

,player,team,line,alt_line,stat,opp,league_id,league_name,event_time,pp_player_id,date,scrape_time,prop_id,event_id,index,id,time,league
3052646,Omarion Hampton,LAC,75.5,standard,Rush+Rec Yds,LV,9.0,NFL,2025-09-15 20:00:00-06:00,244038.0,2025-09-15,2025-09-15 16:10:00-06:00,87be6fec-1,03136d0d-8,NaN,None,None,NaN
3052647,Omarion Hampton,LAC,8.5,standard,Longest Reception,LV,9.0,NFL,2025-09-15 20:00:00-06:00,244038.0,2025-09-15,2025-09-15 16:10:00-06:00,616aa738-b,03136d0d-8,NaN,None,None,NaN
3052648,Hassan Haskins,LAC,0.5,demon,Rush+Rec TDs,LV,9.0,NFL,2025-09-15 20:00:00-06:00,216945.0,2025-09-15,2025-09-15 16:10:00-06:00,598f10d9-8,03136d0d-8,NaN,None,None,NaN
3052649,Omarion Hampton,LAC,13.5,standard,Longest Rush,LV,9.0,NFL,2025-09-15 20:00:00-06:00,244038.0,2025-09-15,2025-09-15 16:10:00-06:00,98856a2f-5,03136d0d-8,NaN,None,None,NaN
3052650,Ashton Jeanty,LV,14.5,standard,Longest Rush,LAC,9.0,NFL,2025-09-15 20:00:00-06:00,246538.0,2025-09-15,2025-09-15 16:10:00-06:00,1e67d0b2-8,03136d0d-8,NaN,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059501,Tyreek Hill,MIA,99.5,demon,Receiving Yards,BUF,9.0,NFL,2025-09-18 18:00:00-06:00,206320.0,2025-09-18,2025-09-15 16:10:00-06:00,18419b56-9,1f3baf69-5,NaN,None,None,NaN
3059502,Tyreek Hill,MIA,24.5,standard,Longest Reception,BUF,9.0,NFL,2025-09-18 18:00:00-06:00,206320.0,2025-09-18,2025-09-15 16:10:00-06:00,1dd3a4bf-2,1f3baf69-5,NaN,None,None,NaN
3059503,Tyreek Hill,MIA,0.5,demon,Rush+Rec TDs,BUF,9.0,NFL,2025-09-18 18:00:00-06:00,206320.0,2025-09-18,2025-09-15 16:10:00-06:00,b7160ba2-a,1f3baf69-5,NaN,None,None,NaN
3059504,James Cook,BUF,1.5,demon,Rush+Rec TDs,MIA,9.0,NFL,2025-09-18 18:00:00-06:00,210711.0,2025-09-18,2025-09-15 16:10:00-06:00,9a660e0d-c,1f3baf69-5,NaN,None,None,NaN


In [23]:
schedule = con.execute('select * from schedule where season = 2025 and week = 2').df()

ConnectionException: Connection Error: Connection already closed!

In [ ]:
today = pd.concat((
    pp.merge(schedule, left_on=['team'], right_on=['home_team']),
    pp.merge(schedule, left_on=['team'], right_on=['away_team'])
))
today['game_line'] = today.apply(lambda x: x['spread_line'] if x['home_team'] == x['team'] else -x['spread_line'],axis=1)
today['favored'] = today['game_line'].apply(lambda x: tiers(x))
today = today[['player','team','stat','alt_line','game_line','favored']].merge(result, on=['stat','alt_line','favored'])
today.loc[today.alt_line=='standard'].sort_values(by='over_ratio').reset_index(drop=True).iloc[0:25]

In [ ]:
today.loc[today.alt_line=='standard'].sort_values(by='over_ratio').reset_index(drop=True).tail(30)

In [ ]:
today.loc[(today.alt_line=='demon') & (today.stat != 'Sacks')].sort_values(by='over_ratio').tail(20)

In [ ]:
result.loc[(result['count'] > 10) &(result['alt_line']=='demon')].sort_values(by=['over_ratio'],ascending=False).drop(['over','push','under'],axis=1).head(30)

In [ ]:
result = pivot_lines(subset, ['alt_line', 'stat', 'pos']).sort_values(by='over_ratio', ascending=False)
result.loc[(result.alt_line == 'standard') & (result['count'] >= 15)].sort_values(by=['over_ratio'],ascending=False).tail(20)

In [ ]:
def get_corr(one, two,three=pd.DataFrame(), grouper=[], relation='same'):
    df1 = one.copy(deep=True)
    df2 = two.copy(deep=True)
    df3 = three.copy(deep=True)
    #get cols to merge on
    grouper = ['alt_line','stat'] + grouper
    home_cols = ['team', 'date']
    opp_cols = ['opp', 'date']
    if relation=='same':
        right_cols = home_cols
    else:
        right_cols = opp_cols

    #fixing weird column renaming
    if not df3.empty:
        a = 4
    else:
        a = 3
    count_cols = []
    o_u_cols = []
    for x in grouper:
        for n in range(1,a):
            count_cols += [x+f'_{n}']
            o_u_cols += [f'o_u_{n}']
    o_u_cols = sorted(list(set(o_u_cols)))

    raw_cols = list(set(grouper + home_cols + right_cols + ['o_u'] + ['player']))
    
    #merge dfs
    comb_corr = df1.loc[(df1.alt_line!='goblin') & (df1.o_u != 'push'), raw_cols].merge(
        df2.loc[(df2.alt_line!='goblin') & (df2.o_u != 'push'), raw_cols],
        left_on=home_cols,
        right_on=right_cols,
        suffixes=('_1', '_2'),
        )
    #filter out same player corr and mirrored lines
    comb_corr = comb_corr.loc[(comb_corr.player_1 != comb_corr.player_2) & 
                              ~((comb_corr.player_1 < comb_corr.player_2) & (comb_corr.stat_1 == comb_corr.stat_2))
                              ].reset_index(drop=True)
    if not df3.empty:
        comb_corr = comb_corr.merge(
            df3.loc[(df3.alt_line!='goblin'), raw_cols],
            left_on=home_cols,
            right_on=right_cols,
            #suffixes=('', '_2'),
            )
        comb_corr.columns = [x+'_3' if (x in raw_cols) else x for x in comb_corr.columns]
        comb_corr = comb_corr.loc[comb_corr.player_1 != comb_corr.player_3].reset_index(drop=True)
        comb_corr = comb_corr.loc[comb_corr.player_2 != comb_corr.player_3].reset_index(drop=True)
    corr = value_count_plus(
        comb_corr,
        count_cols,
        o_u_cols,
    )
    corr['relation'] = relation
    corr = corr.loc[~((corr.alt_line_1.isin(['demon', 'goblin'])) & (corr.o_u_1 == 'under'))]
    corr = corr.loc[~((corr.alt_line_2.isin(['demon', 'goblin'])) & (corr.o_u_2 == 'under'))]
    if not df3.empty:
        corr = corr.loc[~((corr.alt_line_3.isin(['demon', 'goblin'])) & (corr.o_u_3 == 'under'))]

    corr = corr.loc[corr['count'] > 0].sort_values(by='proportion', ascending=False)
    corr['equiv'] = corr['proportion'] ** (1/(a-1))
    corr = corr.rename({'count':'count_all'},axis=1)

    #filtering extraneous permutations
    numbered_cols = []
    for num in range(1,a):
        numbered_cols.append(num)
        corr[num] = ''
        for col in (['stat', 'alt_line', 'o_u'] + grouper):
            num_col = f'{col}_{num}'
            corr[num] += corr[num_col].astype(str)
    if a == 4:
        corr['concat'] = corr.apply(lambda x: str(sorted(list((x[1],x[2],x[3])))),axis=1)
    else:
        corr['concat'] = corr.apply(lambda x: str(sorted(list((x[1],x[2])))),axis=1)
    corr = corr.groupby(['concat']).head(1)
    corr = corr.sort_values(by='equiv',ascending=False).drop([1,2,3,'concat'],axis=1,errors='ignore')

    return corr.sort_values(by='proportion',ascending=False)

def weight_corr(corr, hit_df,suff):
    merge_cols = []
    for col in hit_df.columns:
        if 'hit_' in col:
            hit_col = col
        elif col != ('count') and col != ('proportion'):
            merge_cols.append(col)
    for n in range(1,4):
        n_merge_cols = [col + f'_{n}' for col in merge_cols]
        if f'alt_line_{n}' in corr.columns:
            corr = corr.merge(
                hit_df,
                left_on=n_merge_cols,
                right_on=merge_cols,
                how='left',
                suffixes=[f'_{n-1}',f'_{n}']
            )
            corr.drop(merge_cols, axis=1, inplace=True)
            corr.rename({f'hit_{suff}': f'hit_{suff}_{n}'},axis=1,inplace=True)
    corr['expected'] = 1
    for n in range(1,4):
        col = f'hit_{suff}_{n}'
        if col in corr.columns:
            corr['expected'] = corr['expected'] * corr[col]
    corr['diff'] = corr['proportion'] - corr['expected']
    return corr

hit_stat = value_count_plus(
	df.loc[df.o_u.isin(['under', 'over'])], ['alt_line', 'stat'], ['o_u']
)
#hit_stat['hit_stat'] = hit_stat.apply(lambda x: pad_results(x), axis=1)
hit_stat = hit_stat.rename({'proportion':'hit_stat'},axis=1)
hit_stat.head(3)



In [ ]:
corr = get_corr(df, df,df,  relation='same', grouper=['pos'])
corr = weight_corr(corr,hit_stat,'stat')
corr.loc[(corr.count_all > 55)].head(25)

In [ ]:
corr.loc[(corr.count_all > 55)].head(25)

In [ ]:
corr.loc[((
    (corr.pos_1=='QB') &
    (corr.pos_2=='K')) | 
    (
    (corr.pos_2=='QB') &
    (corr.pos_1=='K')
) )& (corr.count_all > 16)
].sort_values(by='diff',ascending=False).head(20)

In [ ]:
corr2 = get_corr(df, df, relation='same', grouper=['FantPos'])

In [ ]:
corr2.loc[(corr2.count_all > 15) & (corr2.stat_1=='FG') & (corr2.stat_2=='Pass Attempts')].head(25)

In [ ]:
corr2.loc[(corr2.stat_1=='Pass Yards') & (corr2.stat_2=='Fantasy Score') & (corr2.stat_3=='FG')]

In [ ]:
opp = get_corr(df, df, relation='opp',grouper=['FantPos'])
opp = opp.loc[(opp.count_all > 20) & ((opp.FantPos_1 =='QB') | (opp.FantPos_2 == 'QB'))]
opp.loc[(opp.count_all > 35)].head(20)

In [ ]:
wg = weight_corr(opp,hit_stat,'stat')
wg['diff'] = wg['proportion'] - wg['expected']
wg.loc[(wg.count_all > 80) & (wg.alt_line_1=='demon')].sort_values(by='diff',ascending=False).head(30)

In [ ]:
corr = get_corr(df, df, relation='same',grouper=['FantPos'])
corr.loc[(corr.count_all > 40) & (corr.stat_1 >= corr.stat_2)].head(30)
#corr.loc[(corr.count_all > 20) & (corr.stat_1 >= corr.stat_2) & (corr.alt_line_1 == 'demon')].head(30)

In [ ]:
corr2.loc[(corr2.stat_1.str.contains('TD')) & (corr2.stat_3.str.contains('TD')) & (corr2.FantPos_2 == 'K')]

In [ ]:
corr.loc[(corr.count_all > 20)& ((corr.alt_line_1 == 'demon') | (corr.alt_line_2 == 'demon'))].head(15)

In [ ]:
corr2.loc[(corr2.count_all > 25) & (
    (corr2.alt_line_1 == 'demon') |
    (corr2.alt_line_2 == 'demon') |
    (corr2.alt_line_3 == 'demon')) & (corr2.stat_1.str.contains("Fantasy")
)].head(30)

In [ ]:
corr2.loc[corr2.stat_1.str.contains('TD') & (corr2.count_all > 10)]

In [ ]:
corr2.to_csv('../reference_data/NFL/corr/corr3.csv')

In [ ]:
corr2.loc[(corr2.count_all > 35)].head(30)

In [ ]:
base = df[['player','team','line','alt_line','stat','opp','event_time','o_u']]
base

In [ ]:
pair =base.merge(base,on=['team','opp','event_time'])
pair = pair.loc[~(pair.player_x <= pair.player_y)]
pair.loc[(pair.player_x=='Dak Prescott') & (pair.player_y.str.contains('Lamb')) & (pair.stat_x=='Pass Yards') & (pair.stat_y=='Receiving Yards'),['o_u_x','o_u_y']].value_counts().reset_index()

In [ ]:
dk = pd.read_parquet('../Lines/dk/dk_2024_10_5.parquet')
dk.loc[dk['player'].str.contains('Chief',na=False)].head(5)

In [ ]:
play_corr = pair.groupby(['player_x','player_y','stat_x','stat_y','alt_line_x','alt_line_y'])[['o_u_x','o_u_y']].value_counts().reset_index()

In [ ]:
lovers = value_count_plus(
        play_corr.loc[play_corr.alt_line_x=='standard'],
        ['player_x','player_y','stat_x','stat_y','alt_line_x','alt_line_y'],
        ['o_u_x','o_u_y'],
    )

In [ ]:
lovers.loc[(lovers.player_x=='Dak Prescott') & (lovers.player_y.str.contains('Lamb')) & (lovers.stat_x=='Pass Yards') & (lovers.stat_y=='Receiving Yards')]

In [ ]:
pair.loc[(pair.player_x=='Dak Prescott') & (pair.player_y.str.contains('Lamb')) & (pair.stat_x=='Pass Yards') & (pair.stat_y=='Receiving Yards') & 
         (pair.alt_line_x=='standard') & (pair.alt_line_y=='standard')]

In [ ]:
common_cols = [ 'team', 'opp', 'home', 'roof', 'surface', 'weather', 'game_line', 'favored', 'total',
       'season',  'date']
qb_cols =     ['player', 'Pass Completions', 'Pass Attempts', 'Pass Yards',
       'Pass TDs', 'INT', 'Sacks', 'Yards Sacked', 'Longest Pass',
       'Passing Rate',  'snaps', 'pct', 'pos']
wr_cols =     ['player', 'Rec Targets', 'Receptions', 'Receiving Yards', 'Rush+Rec Yds', 
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fantasy Score',  'snaps', 'pct', 'pos']
rb_cols =     ['player', 'Rush Attempts', 'Rush Yards', 'Rush TDs', 'Rush+Rec Yds', 
       'Longest Rush', 'Rec Targets', 'Receptions', 'Receiving Yards',
       'Rec TDs','Rush+Rec TDs', 'Longest Reception', 'Fumbles', 'Fumbles Lost', 'Fantasy Score', 'snaps', 'pct',
        'pos']

k_cols = ['player', 'XPM','Kicking Points', 'Fantasy Score', 'FG','snaps', 'pct', 'pos']

small = games.loc[games['date'] > '2014-09-01']

qb = small.loc[small.pos=='QB'][
    qb_cols + common_cols
]
wr = small.loc[small.pos=='WR'][
    wr_cols + common_cols
]
rb = small.loc[small.pos=='RB'][
    rb_cols + common_cols
]
te = small.loc[small.pos=='TE'][
    wr_cols + common_cols
]
k = small.loc[small.pos=='K'][
    k_cols + common_cols
]

In [ ]:
p1 = 'Jared Goff'
b = small.loc[small.player==p1]

p2 = 'Amon-Ra St. Brown'
t = small.loc[small.player==p2]

both = b.merge(t, on=common_cols)
print(len(both))
both.columns

In [ ]:
import matplotlib.pyplot as plt
stat = 'Pass Attempts'
stat1 = stat + '_x'
stat2 = stat + '_y'
stat2 = 'Rec Targets_y'

plt.axvline(x=both[stat1].median(), label=f'{p1} avg', color='r')
plt.axhline(y=both[stat2].median(), label=f'{p2} avg')
plt.scatter(both[stat1], both[stat2], c=both.index, cmap=plt.cm.coolwarm)
plt.xlabel(p1)
plt.ylabel(p2)
plt.title(stat)
plt.legend()

In [ ]:
both[[stat1,stat2]]